# Rossmann Store Sales



# Bibliotheken importieren



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(rc={'figure.figsize':(20,10)})

C:\tools\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\tools\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\tools\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Daten importieren

In [3]:
import pandas as pd
from pandas.api.types import CategoricalDtype
# pd.set_option('display.max_columns', None)

dtypes_train = {
    'Store' : 'int',
    'DayOfWeek' : 'category',
    'Sales' : 'int',
    'Customers' : 'int',
    'Open' : 'bool',
    'Promo' : 'bool',
    'StateHoliday' : 'category',
    'SchoolHoliday' : 'bool'
}
train = pd.read_csv('train.csv', parse_dates = ['Date'], dtype = dtypes_train)
# display(train.info())
# display(train.head())

dtypes_store = {
    'Store' : 'int',
    'StoreType' : 'category',
    'Assortment' : 'category',
    'CompetitionDistance' : 'object',
    'CompetitionOpenSinceMonth' : 'object',
    'CompetitionOpenSinceYear' : 'object',
    'Promo2' : 'bool',
    'Promo2SinceWeek' : 'object',
    'Promo2SinceYear' : 'object',
    'PromoInterval' : 'object'
}

store = pd.read_csv('store.csv', dtype = dtypes_store)
# display(store.info())
# display(store.head())

demand = train.merge(store, how = 'inner', on = 'Store')
demand.set_index(['Date'], inplace = True)
demand.sort_index(axis = 0, ascending = True, inplace = True)
display(demand.info())
display(demand.head())
# display(daily_demand.info())
# display(daily_demand.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   Store                      1017209 non-null  int32   
 1   DayOfWeek                  1017209 non-null  category
 2   Sales                      1017209 non-null  int32   
 3   Customers                  1017209 non-null  int32   
 4   Open                       1017209 non-null  bool    
 5   Promo                      1017209 non-null  bool    
 6   StateHoliday               1017209 non-null  category
 7   SchoolHoliday              1017209 non-null  bool    
 8   StoreType                  1017209 non-null  category
 9   Assortment                 1017209 non-null  category
 10  CompetitionDistance        1014567 non-null  object  
 11  CompetitionOpenSinceMonth  693861 non-null   object  
 12  CompetitionOpenSinceYear   693861 non-nul

None

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1115,2,0,0,False,False,a,True,d,c,5350,NaN,NaN,True,22,2012,"Mar,Jun,Sept,Dec"
2013-01-01,746,2,0,0,False,False,a,True,d,c,4330,2,2011,True,35,2011,"Mar,Jun,Sept,Dec"
2013-01-01,171,2,0,0,False,False,a,True,a,a,2640,NaN,NaN,False,NaN,NaN,NaN
2013-01-01,694,2,0,0,False,False,a,True,a,c,460,11,2012,True,40,2014,"Jan,Apr,Jul,Oct"
2013-01-01,396,2,0,0,False,False,a,True,a,c,23130,NaN,NaN,False,NaN,NaN,NaN


# Daten vorbereiten

In [4]:
demand['StateHoliday'] = demand['StateHoliday'].replace(to_replace = ['0','a','b','c'], value = ['None', 'Public holiday', 'Easter', 'Christmas']).astype('category')
demand['Assortment'] = demand['Assortment'].replace(to_replace = ['a','b','c'], value = ['basic', 'extra', 'extended']).astype('category')
demand['DayOfWeek'] = demand['DayOfWeek'].replace(to_replace = ['1','2','3','4','5','6','7'], value = ['Monday', 'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']).astype('category')

display(demand.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   Store                      1017209 non-null  int32   
 1   DayOfWeek                  1017209 non-null  category
 2   Sales                      1017209 non-null  int32   
 3   Customers                  1017209 non-null  int32   
 4   Open                       1017209 non-null  bool    
 5   Promo                      1017209 non-null  bool    
 6   StateHoliday               1017209 non-null  category
 7   SchoolHoliday              1017209 non-null  bool    
 8   StoreType                  1017209 non-null  category
 9   Assortment                 1017209 non-null  category
 10  CompetitionDistance        1014567 non-null  object  
 11  CompetitionOpenSinceMonth  693861 non-null   object  
 12  CompetitionOpenSinceYear   693861 non-nul

None

## Modellieren und Schätzen

### Prophet

In [25]:
#!pip install prophet
from prophet import Prophet
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as MAE

data = pd.DataFrame(demand[demand.Store ==1]['Sales'])
data.reset_index(inplace = True)
data.columns = ['ds', 'y']
# display(data)

train, test = temporal_train_test_split(demand[demand.Store ==1], test_size = 28)
#display(test)

state_holidays = pd.DataFrame(demand[demand.StateHoliday != 'None']['StateHoliday'])
state_holidays.reset_index(inplace = True)
state_holidays.columns = ['ds', 'holiday']
state_holidays['lower_window'] = 0
state_holidays['upper_window'] = 0

display(state_holidays)

school_holidays = pd.DataFrame(demand[demand.SchoolHoliday == True]['SchoolHoliday'])
school_holidays.reset_index(inplace = True)
school_holidays.columns = ['ds', 'holiday']
school_holidays['holiday'] = 'School Holiday'
school_holidays['lower_window'] = 0
school_holidays['upper_window'] = 0

display(school_holidays)

holidays = pd.concat((state_holidays, school_holidays))
display(holidays)

data['Open'] = demand[demand.Store ==1]['Open'].astype('int').to_numpy()
data['Promo'] = demand[demand.Store ==1]['Promo'].astype('int').to_numpy()

display(data)

p = Prophet(yearly_seasonality = True, holidays = holidays)
p.add_regressor('Open')
p.add_regressor('Promo')
p.fit(data)

forecast = p.make_future_dataframe(periods = len(test), freq='D', include_history = False)
forecast['Open'] = test['Open'].to_numpy()
forecast['Promo'] = test['Promo'].to_numpy()
display(forecast)

prediction = p.predict(forecast)

print('MAE:\t %4.2f' % (MAE(y_true = test['Sales'], y_pred = prediction)))

display(prediction)

fig = p.plot(forecast)
fig = p.plot_components(forecast)

,ds,holiday,lower_window,upper_window
0,2013-01-01,Public holiday,0,0
1,2013-01-01,Public holiday,0,0
2,2013-01-01,Public holiday,0,0
3,2013-01-01,Public holiday,0,0
4,2013-01-01,Public holiday,0,0
...,...,...,...,...
31045,2015-06-04,Public holiday,0,0
31046,2015-06-04,Public holiday,0,0
31047,2015-06-04,Public holiday,0,0
31048,2015-06-04,Public holiday,0,0


,ds,holiday,lower_window,upper_window
0,2013-01-01,School Holiday,0,0
1,2013-01-01,School Holiday,0,0
2,2013-01-01,School Holiday,0,0
3,2013-01-01,School Holiday,0,0
4,2013-01-01,School Holiday,0,0
...,...,...,...,...
181716,2015-07-31,School Holiday,0,0
181717,2015-07-31,School Holiday,0,0
181718,2015-07-31,School Holiday,0,0
181719,2015-07-31,School Holiday,0,0


,ds,holiday,lower_window,upper_window
0,2013-01-01,Public holiday,0,0
1,2013-01-01,Public holiday,0,0
2,2013-01-01,Public holiday,0,0
3,2013-01-01,Public holiday,0,0
4,2013-01-01,Public holiday,0,0
...,...,...,...,...
181716,2015-07-31,School Holiday,0,0
181717,2015-07-31,School Holiday,0,0
181718,2015-07-31,School Holiday,0,0
181719,2015-07-31,School Holiday,0,0


,ds,y,Open,Promo
0,2013-01-01,0,0,0
1,2013-01-02,5530,1,0
2,2013-01-03,4327,1,0
3,2013-01-04,4486,1,0
4,2013-01-05,4997,1,0
...,...,...,...,...
937,2015-07-27,6102,1,1
938,2015-07-28,5011,1,1
939,2015-07-29,4782,1,1
940,2015-07-30,5020,1,1


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


KeyboardInterrupt: 